In [118]:
import numpy as np
import pandas as pd
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib
import networkx as nx
import pickle
from collections import OrderedDict
import copy
from scipy.sparse import csr_matrix
from scipy import io
import seaborn as sns
import joblib
from base import *
from joblib import Parallel, delayed
import random
import scipy as sc
import sys

In [119]:
from itertools import islice
def k_shortest_paths(G, source, target, k, weight=None):
    def path_cost(G, path):
        return sum([G[path[i]][path[i+1]]['length'] for i in range(len(path)-1)])
    p = list(islice(nx.shortest_simple_paths(G, source, target, weight=weight), k))
    #print('p',p)
#     if(len(p) < k) or (len(p[k-1]) <= 0):
#         return (float('inf'), [])
#     else:
#         return [path_cost(G,p[k-1]), p[k-1]]
    return p

#print(k_shortest_paths(G,737184,718421,1,weight='length'))

## Set parameters

In [120]:
cur_date_time = datetime.date(2020, 3, 20)

vector_2019_path = './'
vector_2020_path = './'

## Read in q vector

In [122]:
# q_vector_2019_path = './Numerical_result/07_04_2022_NumericalResult/OD_03_20_03_20_2020_netwrokL_beta0'
# q_vector_2020_path = './Numerical_result/07_04_2022_NumericalResult/OD_03_20_03_20_2020_netwrokL_beta0'

# read in Q_vector

# q_vector_folder_2019 = q_vector_2019_path + '/Q_vector/q_vector.npy'
# q_vector_folder_2020 = q_vector_2020_path + '/Q_vector/q_vector.npy'

q_vector_folder_2019 = vector_2019_path + '/Q_vector/q_vector_sym_1e3.npy'
q_vector_folder_2020 = vector_2020_path + '/Q_vector/q_vector_sym_1e3.npy'


# q_vector_2019 = np.load(q_vector_folder_2019)
q_est = np.load(q_vector_folder_2020)

time_min_interval = 6*60
N = int(60 / time_min_interval * 24)-1
print(N)

# N = 146
q_vector_2d_2019 = q_vector_2019.reshape((N,-1))
q_vector_2d_2020 = q_vector_2020.reshape((N,-1))
print(q_vector_2d_2020.shape)

3
(3, 1806)


## Read in A matrix and b vector

In [123]:
single_date = cur_date_time
date_str = single_date.strftime("%Y-%m-%d")

x_o = np.load(os.path.join(vector_2019_path, 'X_vector', date_str + ".npy"))
print('x: ',len(x_o))

r = joblib.load(os.path.join(vector_2019_path, "R_matrix", date_str + ".pickle")).tocsr()
print('r: ', r.shape)

P = joblib.load(os.path.join(vector_2019_path, "P_matrix", date_str+".pickle")).tocsr()
print('P: ',P.shape)

observe_index_N = np.load(vector_2019_path+"observe_index_N.npy")

RP = np.array(r.dot(P).toarray())[observe_index_N == 1,:]

Alr_sym = np.load(os.path.join(vector_2019_path, 'Constraints', "Alr_sym.npy"))
Alr_arterial = np.load(os.path.join(vector_2019_path, 'Constraints', "Alr_arterial.npy"))

b_sym = np.load(os.path.join(vector_2019_path, 'Constraints', "b_sym.npy"))
b_arterial = np.load(os.path.join(vector_2019_path, 'Constraints', "b_arterial.npy"))


x:  390
r:  (390, 5418)
P:  (5418, 5418)


## Calculate errors

In [116]:
q_est_new = q_est + 75

print(q_est)
print(q_est_new)
x_est =  RP.dot(q_est_new)
        
error1 = np.sum(np.absolute(x_o - x_est))
error2 = np.sum(np.absolute(Alr_sym.dot(q_est_new)))
error3 = np.sum(np.absolute(Alr_arterial[:,:q_est.shape[0]].dot(q_est)))
# print(Alr_sym.dot(q_est_new))
print('error1, ',error1,'error2, ',error2,'error3, ',error3)

[0.48922333 0.65858567 1.3186023  ... 3.6099029  2.438229   2.5893772 ]
[75.48922  75.658585 76.3186   ... 78.6099   77.43823  77.58938 ]
error1,  5060913.958485582 error2,  497.40174102783203 error3,  9635.361413151026


In [117]:
Alr_arterial.shape

(21, 5439)

## Calculate available path

In [4]:
import importlib
importlib.reload(sys.modules['base'])

start_month=3
start_day=1
end_month=3
end_day=10
year =2020
flag_weekday = 1

read_in_folder = '../Raw_data/Processed_data/highway_' + str(year) + '/month_'+ str(start_month) +'_'+str(year)+'/'

with open(read_in_folder+'od_list.pickle', 'rb') as handle:
    (O_list, D_list) = pickle.load(handle)
# with open('graph.pickle', 'rb') as handle:
#     G = pickle.load(handle)
G = nx.read_gpickle(read_in_folder+'graph_4.pickle')

OD_paths = OrderedDict()
link_dict = OrderedDict()
path_list = list()
# O_list = O_list[:]
# D_list = D_list[:]
shortest_path_list = list()
count = 0;
k=0
N_path = 1
# while len(shortest_path_list) < N_path:
OD_list_output = []
print(O_list)
for O in O_list:
    for D in O_list:
        print(O, D);
        if O!=D:
            try:
                paths = k_shortest_paths(G,O, D,N_path,weight='length') #My code
                print('paths, ',paths)
                if len(paths) != 0:
                    tmp_path_list = list()
                    for path in paths:
                        path_o = Path()
                        path_o.node_list = path
                        path_o.node_to_list(G, link_dict)
                        print(path_o.cost)
                        tmp_path_list.append(path_o)
                        path_list.append(path_o)
                        
                    OD_paths[(O, D)] = tmp_path_list
                    shortest_path_list.append(paths)
                    OD_list_output.append([O,D])
                    print("From ", O, " To ", D, "there are ", N_path, "paths")
                else:
                    pass
            except:
#                     print("From ", O, " To ", D, "there are no paths")
                pass
        else:
#                 print("From ", O, " To ", D, "same")
            pass

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
0 0
0 1
paths,  [[0, 1]]
5.911740321184936
From  0  To  1 there are  1 paths
0 2
paths,  [[0, 1, 41, 33, 34, 37, 2]]
46.71688251917065
From  0  To  2 there are  1 paths
0 3
paths,  [[0, 1, 41, 33, 34, 36, 22, 3]]
63.969360149523936
From  0  To  3 there are  1 paths
0 4
paths,  [[0, 1, 16, 7, 4]]
26.583642382959077
From  0  To  4 there are  1 paths
0 5
paths,  [[0, 1, 16, 7, 4, 5]]
36.02129478781398
From  0  To  5 there are  1 paths
0 6
paths,  [[0, 1, 16, 7, 4, 5, 6]]
42.74119829180726
From  0  To  6 there are  1 paths
0 7
paths,  [[0, 1, 16, 7]]
18.747222313739826
From  0  To  7 there are  1 paths
0 8
paths,  [[0, 1, 16, 7, 10, 8]]
40.69071460251563
From  0  To  8 there are  1 paths
0 9
paths,  [[0, 1, 41, 33, 32, 31, 9]]
34.63644753966257
From  0  To  9 there are  1 paths
0 10
paths,  [[0, 1, 16, 7, 10]]
30.445458843425776


4 37
paths,  [[4, 38, 33, 34, 37]]
23.175991716721775
From  4  To  37 there are  1 paths
4 38
paths,  [[4, 38]]
5.842456537989821
From  4  To  38 there are  1 paths
4 39
paths,  [[4, 7, 16, 1, 39]]
26.47716383837961
From  4  To  39 there are  1 paths
4 40
paths,  [[4, 38, 33, 32, 40]]
24.047991749489903
From  4  To  40 there are  1 paths
4 41
paths,  [[4, 38, 33, 41]]
19.26562276888648
From  4  To  41 there are  1 paths
4 42
paths,  [[4, 5, 42]]
14.420315322433094
From  4  To  42 there are  1 paths
5 0
paths,  [[5, 4, 7, 16, 1, 0]]
36.02129478781398
From  5  To  0 there are  1 paths
5 1
paths,  [[5, 4, 7, 16, 1]]
30.109554466629042
From  5  To  1 there are  1 paths
5 2
paths,  [[5, 6, 18, 22, 2]]
26.01435249078797
From  5  To  2 there are  1 paths
5 3
paths,  [[5, 6, 18, 22, 3]]
42.44680911128302
From  5  To  3 there are  1 paths
5 4
paths,  [[5, 4]]
9.4376524048549
From  5  To  4 there are  1 paths
5 5
5 6
paths,  [[5, 6]]
6.719903503993278
From  5  To  6 there are  1 paths
5 7
paths,

From  9  To  21 there are  1 paths
9 22
paths,  [[9, 31, 34, 36, 22]]
31.19580435843632
From  9  To  22 there are  1 paths
9 23
paths,  [[9, 31, 34, 35, 42, 5, 24, 23]]
39.86251868426444
From  9  To  23 there are  1 paths
9 24
paths,  [[9, 31, 34, 35, 42, 5, 24]]
32.930675557984884
From  9  To  24 there are  1 paths
9 25
paths,  [[9, 31, 30, 19, 25]]
24.148363894915228
From  9  To  25 there are  1 paths
9 26
paths,  [[9, 31, 30, 26]]
16.031261030931514
From  9  To  26 there are  1 paths
9 27
paths,  [[9, 31, 30, 19, 29, 27]]
46.30548684979625
From  9  To  27 there are  1 paths
9 28
paths,  [[9, 31, 30, 19, 29, 27, 28]]
88.865417024338
From  9  To  28 there are  1 paths
9 29
paths,  [[9, 31, 30, 19, 29]]
31.1492717750059
From  9  To  29 there are  1 paths
9 30
paths,  [[9, 31, 30]]
10.608234363119484
From  9  To  30 there are  1 paths
9 31
paths,  [[9, 31]]
6.998605877910574
From  9  To  31 there are  1 paths
9 32
paths,  [[9, 31, 32]]
10.199727284382359
From  9  To  32 there are  1 pat

From  13  To  32 there are  1 paths
13 33
paths,  [[13, 26, 30, 31, 32, 33]]
22.632869285427635
From  13  To  33 there are  1 paths
13 34
paths,  [[13, 26, 30, 31, 34]]
21.567066388823505
From  13  To  34 there are  1 paths
13 35
paths,  [[13, 26, 30, 31, 34, 35]]
26.193159890897444
From  13  To  35 there are  1 paths
13 36
paths,  [[13, 26, 30, 31, 34, 36]]
27.116219513411906
From  13  To  36 there are  1 paths
13 37
paths,  [[13, 26, 30, 31, 34, 37]]
27.552265586820063
From  13  To  37 there are  1 paths
13 38
paths,  [[13, 26, 30, 31, 32, 33, 38]]
28.936355047159765
From  13  To  38 there are  1 paths
13 39
paths,  [[13, 26, 30, 31, 32, 33, 41, 1, 39]]
43.39566279193921
From  13  To  39 there are  1 paths
13 40
paths,  [[13, 26, 19, 25, 40]]
26.152520508081967
From  13  To  40 there are  1 paths
13 41
paths,  [[13, 26, 30, 31, 32, 33, 41]]
29.752549754592167
From  13  To  41 there are  1 paths
13 42
paths,  [[13, 26, 30, 31, 34, 35, 42]]
32.68677890754987
From  13  To  42 there are 

From  18  To  29 there are  1 paths
18 30
paths,  [[18, 35, 34, 31, 30]]
25.409068918088195
From  18  To  30 there are  1 paths
18 31
paths,  [[18, 35, 34, 31]]
21.799440432879283
From  18  To  31 there are  1 paths
18 32
paths,  [[18, 35, 34, 32]]
20.77126832158517
From  18  To  32 there are  1 paths
18 33
paths,  [[18, 35, 34, 33]]
21.141523937825717
From  18  To  33 there are  1 paths
18 34
paths,  [[18, 35, 34]]
16.09667371882245
From  18  To  34 there are  1 paths
18 35
paths,  [[18, 35]]
11.47058021674851
From  18  To  35 there are  1 paths
18 36
paths,  [[18, 35, 36]]
13.931699423444098
From  18  To  36 there are  1 paths
18 37
paths,  [[18, 35, 36, 37]]
16.944505224898016
From  18  To  37 there are  1 paths
18 38
paths,  [[18, 6, 5, 4, 38]]
24.779008127413032
From  18  To  38 there are  1 paths
18 39
paths,  [[18, 35, 34, 33, 41, 1, 39]]
41.90431744433729
From  18  To  39 there are  1 paths
18 40
paths,  [[18, 35, 34, 32, 40]]
29.10586956716395
From  18  To  40 there are  1 pat

From  23  To  27 there are  1 paths
23 28
paths,  [[23, 10, 7, 16, 1, 39, 15, 27, 28]]
110.50641494531015
From  23  To  28 there are  1 paths
23 29
paths,  [[23, 24, 5, 42, 35, 34, 32, 40, 25, 29]]
56.11848658759492
From  23  To  29 there are  1 paths
23 30
paths,  [[23, 24, 5, 42, 35, 34, 31, 30]]
36.473541291562775
From  23  To  30 there are  1 paths
23 31
paths,  [[23, 24, 5, 42, 35, 34, 31]]
32.863912806353866
From  23  To  31 there are  1 paths
23 32
paths,  [[23, 24, 5, 42, 35, 34, 32]]
31.835740695059755
From  23  To  32 there are  1 paths
23 33
paths,  [[23, 10, 11, 4, 38, 33]]
28.998994071078318
From  23  To  33 there are  1 paths
23 34
paths,  [[23, 24, 5, 42, 35, 34]]
27.161146092297034
From  23  To  34 there are  1 paths
23 35
paths,  [[23, 24, 5, 42, 35]]
22.535052590223096
From  23  To  35 there are  1 paths
23 36
paths,  [[23, 24, 5, 42, 35, 36]]
24.996171796918684
From  23  To  36 there are  1 paths
23 37
paths,  [[23, 24, 5, 42, 35, 36, 37]]
28.008977598372603
From  23

28 24
paths,  [[28, 27, 29, 25, 40, 32, 34, 35, 42, 5, 24]]
106.90278871064747
From  28  To  24 there are  1 paths
28 25
paths,  [[28, 27, 29, 25]]
67.07584445641768
From  28  To  25 there are  1 paths
28 26
paths,  [[28, 27, 29, 19, 26]]
77.4494836758882
From  28  To  26 there are  1 paths
28 27
paths,  [[28, 27]]
42.55993017454175
From  28  To  27 there are  1 paths
28 28
28 29
paths,  [[28, 27, 29]]
57.7161452493321
From  28  To  29 there are  1 paths
28 30
paths,  [[28, 27, 29, 19, 30]]
78.25718266121851
From  28  To  30 there are  1 paths
28 31
paths,  [[28, 27, 29, 19, 30, 31]]
81.86681114642742
From  28  To  31 there are  1 paths
28 32
paths,  [[28, 27, 29, 25, 40, 32]]
81.99889114186726
From  28  To  32 there are  1 paths
28 33
paths,  [[28, 27, 29, 25, 40, 32, 33]]
85.56633934605644
From  28  To  33 there are  1 paths
28 34
paths,  [[28, 27, 29, 25, 40, 32, 34]]
86.67348574462999
From  28  To  34 there are  1 paths
28 35
paths,  [[28, 27, 29, 25, 40, 32, 34, 35]]
91.2995792467

From  33  To  21 there are  1 paths
33 22
paths,  [[33, 34, 36, 22]]
23.53928198547218
From  33  To  22 there are  1 paths
33 23
paths,  [[33, 38, 4, 11, 10, 23]]
28.998994071078318
From  33  To  23 there are  1 paths
33 24
paths,  [[33, 34, 35, 42, 5, 24]]
25.274153185020747
From  33  To  24 there are  1 paths
33 25
paths,  [[33, 32, 40, 25]]
18.490494889638747
From  33  To  25 there are  1 paths
33 26
paths,  [[33, 32, 31, 30, 26]]
15.801224763681896
From  33  To  26 there are  1 paths
33 27
paths,  [[33, 32, 40, 25, 29, 27]]
43.00640917151469
From  33  To  27 there are  1 paths
33 28
paths,  [[33, 32, 40, 25, 29, 27, 28]]
85.56633934605644
From  33  To  28 there are  1 paths
33 29
paths,  [[33, 32, 40, 25, 29]]
27.850194096724337
From  33  To  29 there are  1 paths
33 30
paths,  [[33, 32, 31, 30]]
10.378198095869866
From  33  To  30 there are  1 paths
33 31
paths,  [[33, 32, 31]]
6.768569610660957
From  33  To  31 there are  1 paths
33 32
paths,  [[33, 32]]
3.567448204189172
From  3

From  38  To  18 there are  1 paths
38 19
paths,  [[38, 33, 32, 31, 30, 19]]
22.440483723479847
From  38  To  19 there are  1 paths
38 20
paths,  [[38, 33, 34, 37, 2, 20]]
39.37086880928987
From  38  To  20 there are  1 paths
38 21
paths,  [[38, 4, 5, 6, 21]]
26.15958020780259
From  38  To  21 there are  1 paths
38 22
paths,  [[38, 33, 34, 36, 22]]
29.842767747204306
From  38  To  22 there are  1 paths
38 23
paths,  [[38, 4, 11, 10, 23]]
22.695508309346188
From  38  To  23 there are  1 paths
38 24
paths,  [[38, 4, 5, 24]]
19.40703647255764
From  38  To  24 there are  1 paths
38 25
paths,  [[38, 33, 32, 40, 25]]
24.793980651370877
From  38  To  25 there are  1 paths
38 26
paths,  [[38, 33, 32, 31, 30, 26]]
22.104710525414028
From  38  To  26 there are  1 paths
38 27
paths,  [[38, 33, 32, 40, 25, 29, 27]]
49.309894933246824
From  38  To  27 there are  1 paths
38 28
paths,  [[38, 33, 32, 40, 25, 29, 27, 28]]
91.86982510778857
From  38  To  28 there are  1 paths
38 29
paths,  [[38, 33, 32,

## Calculate traffic flow between health district

In [6]:
## calculate traffic flow between health district
## 1. read in all path file
node_df = pd.read_csv('./processed_data/node_list.csv')
dict_hd = pd.read_csv('./processed_data/hd_centroid.csv',sep=',')[['HD_ID', 'HD_NAME']]
dict_hd = dict_hd.sort_values(by='HD_ID', ascending=False).groupby('HD_ID').first().reset_index(drop = False)

path_flow = np.zeros((26,26))

count = 1

vol_link_est = np.zeros((N,len(link_dict)))
print(vol_link_est.shape)
for idx, i in enumerate(OD_paths.values()):
    for ii in i:
        ii.q = q_vector_2d_2020[:,idx]
        

for key, value in OD_paths.items():
    q_sum = value[0].q.sum()
#     print('From ',from_region,' to ', to_region, ': ', q_sum)
    from_region = node_df.loc[key[0], 'hd']
    to_region = node_df.loc[key[1], 'hd']
    
#     print('from_region, ', from_region)
    path_flow[from_region-1,to_region-1] += q_sum
    
    count += 1
#     print(count)
# print(path_flow)
path_flow = np.round(path_flow, 1)
df = pd.DataFrame(path_flow, columns = dict_hd['HD_NAME'].to_numpy(), index = dict_hd['HD_NAME'].to_numpy())
df.loc['Sum']= df.sum(numeric_only=True, axis=0)
df.loc[:,'Sum'] = df.sum(numeric_only=True, axis=1)


# path_flow[:,27] = path_flow.sum(axis = 0)
# path_flow[27,:] = path_flow.sum(axis = 1)

# path_flow[1:27,0] = dict_hd['HD_NAME'].to_numpy()
# path_flow[0,1:27] = dict_hd['HD_NAME'].to_numpy()

# print(q_vector_2020)
df_path = q_vector_2019_path + '/path_flow_2020_new_head_1e10.csv'
df.to_csv(df_path)
q_path = q_vector_2019_path + '/path_flow_2020_new_1e10.csv'
np.savetxt(q_path,path_flow)

# for idx, i in enumerate(OD_paths.values()):
#     for ii in i:
#         ii.q = q_vector_2d_2020[:,idx]
#         print(ii.q.shape)
#         for iii in ii.link_list:
#             vol_link_est[:,iii.ID] += ii.q/ii.cost *iii.length
#             print(vol_link_est)
#             print(iii.ID,' ', end = '')
#         print('')

(3, 130)


## Calculate difference along symmetric

In [75]:
path_flow = np.round(path_flow, 1)

print(path_flow.shape)

path_flow=np.append(path_flow,[np.sum(path_flow,axis=0)],axis=0)

col=np.array([np.sum(path_flow,axis=1)])
path_flow=np.concatenate((path_flow,col.T),axis=1)


path_sym_diff = abs(path_flow.T - path_flow)

# df_1 = pd.DataFrame(path_flow, columns = df['HD_NAME'].to_numpy(), index = dict_hd['HD_NAME'].to_numpy())
# df.loc['Sum']= df.sum(numeric_only=True, axis=0)
# df.loc[:,'Sum'] = df.sum(numeric_only=True, axis=1)
path_sym_diff = np.round(path_sym_diff, 1)
np.savetxt(q_vector_2019_path + '/path_sym_diff_1e10.csv',path_sym_diff)
df[:] = path_sym_diff

df.to_csv(q_vector_2019_path + '/path_sym_diff_head_1e10.csv')

print(path_sym_diff)

(26, 26)
[[0.000e+00 4.000e-01 4.000e-01 3.000e-01 3.000e-01 3.000e-01 1.800e+00
  1.000e-01 2.000e-01 0.000e+00 6.000e-01 3.000e-01 4.000e-01 1.200e+00
  5.000e-01 2.000e-01 2.000e-01 0.000e+00 2.800e+00 1.100e+00 5.000e-01
  1.300e+00 3.000e-01 5.000e-01 1.700e+00 3.000e-01 3.700e+00]
 [4.000e-01 0.000e+00 3.000e-01 3.000e-01 2.500e+00 2.000e-01 4.400e+00
  1.000e-01 2.000e-01 3.000e-01 3.000e-01 1.000e-01 1.000e-01 2.290e+01
  1.400e+00 0.000e+00 0.000e+00 7.000e-01 6.400e+00 3.000e+00 1.400e+00
  3.100e+00 0.000e+00 4.000e-01 6.100e+00 0.000e+00 4.680e+01]
 [4.000e-01 3.000e-01 0.000e+00 2.000e-01 1.500e+00 1.000e+00 2.000e+00
  2.000e-01 1.000e-01 4.000e-01 3.000e-01 1.000e-01 1.400e+00 3.000e-01
  2.000e-01 3.000e-01 3.000e-01 1.000e+00 3.900e+00 6.000e-01 2.000e-01
  1.800e+00 4.000e-01 6.000e-01 2.600e+00 3.000e-01 8.400e+00]
 [3.000e-01 3.000e-01 2.000e-01 0.000e+00 2.000e-01 0.000e+00 4.000e+00
  1.000e-01 2.000e-01 2.000e-01 0.000e+00 3.000e-01 2.000e-01 1.100e+00
  1.200e+0